In [1]:
import pandas as pd # DataFrame operations plus .csv I/O
import numpy as np # linear algebra
import os

nypd_df1 = pd.read_csv('NYPD_Arrest_Data__Year_to_Date_.csv')
nypd_df1.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,220756993,11/19/2020,155.0,RAPE 2,104.0,RAPE,PL 1303001,F,B,41,0,18-24,M,BLACK,1013232,236725,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
1,220422940,11/12/2020,157.0,RAPE 1,104.0,RAPE,PL 1303502,F,Q,112,0,25-44,M,BLACK,1025420,202485,40.722364,-73.851474,POINT (-73.85147389399998 40.72236368700004)
2,218804160,10/06/2020,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,M,7,2,25-44,M,BLACK,988708,200317,40.716508,-73.983920,POINT (-73.98391989899994 40.71650772200008)
3,218641095,10/02/2020,594.0,OBSCENITY 1,116.0,SEX CRIMES,PL 2631600,F,M,5,0,25-44,M,WHITE,984946,200203,40.716196,-73.997491,POINT (-73.99749074599998 40.716195914000025)
4,217890704,09/16/2020,155.0,RAPE 2,104.0,RAPE,PL 1303001,F,K,77,0,25-44,M,WHITE HISPANIC,1003606,185050,40.674583,-73.930222,POINT (-73.93022154099998 40.67458330800008)


Great! Let's see how many columns and rows we have.

In [2]:
nypd_df1.columns.values

array(['ARREST_KEY', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD',
       'OFNS_DESC', 'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO',
       'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX',
       'PERP_RACE', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'New Georeferenced Column'], dtype=object)

In [3]:
nypd_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140413 entries, 0 to 140412
Data columns (total 19 columns):
ARREST_KEY                  140413 non-null int64
ARREST_DATE                 140413 non-null object
PD_CD                       140390 non-null float64
PD_DESC                     140376 non-null object
KY_CD                       140376 non-null float64
OFNS_DESC                   140376 non-null object
LAW_CODE                    140413 non-null object
LAW_CAT_CD                  139024 non-null object
ARREST_BORO                 140413 non-null object
ARREST_PRECINCT             140413 non-null int64
JURISDICTION_CODE           140413 non-null int64
AGE_GROUP                   140413 non-null object
PERP_SEX                    140413 non-null object
PERP_RACE                   140413 non-null object
X_COORD_CD                  140413 non-null int64
Y_COORD_CD                  140413 non-null int64
Latitude                    140413 non-null float64
Longitude               

Looks like we have 140413 arrests by NYPD in 2020. I want to see if there are any null values and decide if we should delete any rows or unnecessary attributes(columns).

In [4]:
nypd_df1.isnull().sum()

ARREST_KEY                     0
ARREST_DATE                    0
PD_CD                         23
PD_DESC                       37
KY_CD                         37
OFNS_DESC                     37
LAW_CODE                       0
LAW_CAT_CD                  1389
ARREST_BORO                    0
ARREST_PRECINCT                0
JURISDICTION_CODE              0
AGE_GROUP                      0
PERP_SEX                       0
PERP_RACE                      0
X_COORD_CD                     0
Y_COORD_CD                     0
Latitude                       0
Longitude                      0
New Georeferenced Column       0
dtype: int64

Hmm looks like we have 5 columns with many null values. Here are the descriptions for those columns according to NYC OpenData (source of data set):


*PD_CD - Three digit internal classification code (more granular than Key Code)

*PD_DESC - Description of internal classification corresponding with PD code (more granular than Offense Description)

*KY_CD - Three digit internal classification code (more general category than PD code)

*OFNS_DESC - Description of internal classification corresponding with KY code (more general category than PD description)

*LAW_CAT_CD - Level of offense: felony, misdemeanor, violation


We should keep LAW_CAT_CD and OFNS_DESC since they look important for this analysis. The other 3 columns will be dropped since they seem to have more meaning to internal users. Also I will get rid of ARREST_KEY because it is likely a primary key for a table and not needed for our analysis. LAW_CODE will also be removed since it appears to be a code familiar to internal users but not needed for analysis.

In [5]:
columns_dropped = ['ARREST_KEY', 'PD_CD', 'KY_CD', 'LAW_CODE', 'X_COORD_CD', 'Y_COORD_CD', 'New Georeferenced Column']
nypd_df1.drop(columns = columns_dropped, inplace = True)
nypd_df1.isnull().sum()

ARREST_DATE             0
PD_DESC                37
OFNS_DESC              37
LAW_CAT_CD           1389
ARREST_BORO             0
ARREST_PRECINCT         0
JURISDICTION_CODE       0
AGE_GROUP               0
PERP_SEX                0
PERP_RACE               0
Latitude                0
Longitude               0
dtype: int64

I removed the unneccessary coordinate data since latitude and longitude is sufficient for what we're doing.

So we still have some nulls to address. For PD_DESC and OFNS_DESC, we can probably impute a value such as 'Unknown'. I'd rather do this than delete the rows entirely.

In [6]:
impute_values = {'PD_DESC': 'UNKNOWN', 'OFNS_DESC': 'UNKNOWN'}
nypd_df1.fillna(value=impute_values, inplace = True)
nypd_df1.isnull().sum()

ARREST_DATE             0
PD_DESC                 0
OFNS_DESC               0
LAW_CAT_CD           1389
ARREST_BORO             0
ARREST_PRECINCT         0
JURISDICTION_CODE       0
AGE_GROUP               0
PERP_SEX                0
PERP_RACE               0
Latitude                0
Longitude               0
dtype: int64

In [7]:
PD_DESC_na = nypd_df1.loc[nypd_df1['PD_DESC'] == 'UNKNOWN'] #check if imputed values were inserted 
PD_DESC_na.head()

,ARREST_DATE,PD_DESC,OFNS_DESC,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
6,07/10/2020,UNKNOWN,UNKNOWN,M,M,17,0,45-64,M,WHITE,40.748786,-73.971807
374,11/11/2020,UNKNOWN,UNKNOWN,M,Q,110,0,25-44,M,BLACK HISPANIC,40.749877,-73.862727
2210,12/12/2020,UNKNOWN,UNKNOWN,F,M,23,0,25-44,M,WHITE HISPANIC,40.787567,-73.943132
2898,11/22/2020,UNKNOWN,UNKNOWN,M,K,62,0,18-24,M,WHITE HISPANIC,40.605724,-74.009192
3589,11/18/2020,UNKNOWN,UNKNOWN,M,M,26,2,25-44,M,WHITE HISPANIC,40.812116,-73.955170


Sweet! I can see PD_DESC and OFNS_DESC appear to have no missing values now. As for LAW_CAT_CD, we will have to do something about the 1389 missing values. The possible values for this column are 'F'(Felony), 'M'(Misdemeanor), and 'I'(infraction/violation). Let's put 'U' for the nulls in this column. 

In [8]:
impute_LAW_CAT = {'LAW_CAT_CD': 'U'}
nypd_df1.fillna(value=impute_LAW_CAT, inplace = True)
nypd_df1.isnull().sum()

ARREST_DATE          0
PD_DESC              0
OFNS_DESC            0
LAW_CAT_CD           0
ARREST_BORO          0
ARREST_PRECINCT      0
JURISDICTION_CODE    0
AGE_GROUP            0
PERP_SEX             0
PERP_RACE            0
Latitude             0
Longitude            0
dtype: int64

Yay no more missing values! Let's check out the cases with LAW_CAT_CD = 'U' just to make sure. 

In [9]:
LAW_CAT_na = nypd_df1.loc[nypd_df1['LAW_CAT_CD'] == 'U'] 
LAW_CAT_na.head() 

,ARREST_DATE,PD_DESC,OFNS_DESC,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
65,12/23/2020,U.S. CODE UNCLASSIFIED,FOR OTHER AUTHORITIES,U,Q,113,0,25-44,F,BLACK,40.665889,-73.801838
89,12/30/2020,U.S. CODE UNCLASSIFIED,FOR OTHER AUTHORITIES,U,B,52,0,25-44,M,BLACK,40.869407,-73.879998
107,12/16/2020,U.S. CODE UNCLASSIFIED,FOR OTHER AUTHORITIES,U,M,5,0,45-64,F,WHITE,40.714865,-74.003158
121,12/08/2020,U.S. CODE UNCLASSIFIED,FOR OTHER AUTHORITIES,U,K,84,0,25-44,M,BLACK,40.697970,-73.989933
280,12/10/2020,U.S. CODE UNCLASSIFIED,FOR OTHER AUTHORITIES,U,M,5,0,25-44,M,BLACK,40.714006,-74.002083


Now I want to rename some columns so they are easier to understand.

In [10]:
renamed_columns = {'PD_DESC': 'PD_Crime_Name', 'OFNS_DESC': 'OFFENSE', 'LAW_CAT_CD': 'Classification'}
nypd_df1.rename(columns= renamed_columns, inplace = True)
nypd_df1.columns.values

array(['ARREST_DATE', 'PD_Crime_Name', 'OFFENSE', 'Classification',
       'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP',
       'PERP_SEX', 'PERP_RACE', 'Latitude', 'Longitude'], dtype=object)

Next I want to add a column for month. We will extract a numerical value for month from ARREST_DATE and create a new column. Then we will change the values to correspond with their month abbreviations.

In [11]:
nypd_df1['arrest_month'] = pd.DatetimeIndex(nypd_df1['ARREST_DATE']).month
renamed_cols = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'June', 7: 'July', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
nypd_df1['arrest_month'].replace(renamed_cols, inplace = True)
nypd_df1.head()

,ARREST_DATE,PD_Crime_Name,OFFENSE,Classification,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude,arrest_month
0,11/19/2020,RAPE 2,RAPE,F,B,41,0,18-24,M,BLACK,40.816392,-73.895296,Nov
1,11/12/2020,RAPE 1,RAPE,F,Q,112,0,25-44,M,BLACK,40.722364,-73.851474,Nov
2,10/06/2020,RAPE 1,RAPE,F,M,7,2,25-44,M,BLACK,40.716508,-73.983920,Oct
3,10/02/2020,OBSCENITY 1,SEX CRIMES,F,M,5,0,25-44,M,WHITE,40.716196,-73.997491,Oct
4,09/16/2020,RAPE 2,RAPE,F,K,77,0,25-44,M,WHITE HISPANIC,40.674583,-73.930222,Sept


Lastly, let's change ARREST_BORO to their full text instead of a single character. This can clear up confusion between B(Bronx) and K(Brooklyn).

In [12]:
new_cols = {'B': 'Bronx', 'Q': 'Queens', 'M': 'Manhattan', 'K': 'Brooklyn', 'S': 'Staten Island'}
nypd_df1['ARREST_BORO'].replace(new_cols, inplace = True)
nypd_df1.head()

,ARREST_DATE,PD_Crime_Name,OFFENSE,Classification,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude,arrest_month
0,11/19/2020,RAPE 2,RAPE,F,Bronx,41,0,18-24,M,BLACK,40.816392,-73.895296,Nov
1,11/12/2020,RAPE 1,RAPE,F,Queens,112,0,25-44,M,BLACK,40.722364,-73.851474,Nov
2,10/06/2020,RAPE 1,RAPE,F,Manhattan,7,2,25-44,M,BLACK,40.716508,-73.983920,Oct
3,10/02/2020,OBSCENITY 1,SEX CRIMES,F,Manhattan,5,0,25-44,M,WHITE,40.716196,-73.997491,Oct
4,09/16/2020,RAPE 2,RAPE,F,Brooklyn,77,0,25-44,M,WHITE HISPANIC,40.674583,-73.930222,Sept


Awesome! Now we have a clean dataset for our visualizations and further analysis. I'll export this to an excel file and use it for other notebooks. 

In [13]:
nypd_df1.to_csv('clean_NYPD_data.csv', index= False)